In [1]:
from pathlib import Path

from inference_parquet import process_embeddings
from train_linear_model_slim import train_and_save
from active_learning_from_parquet import plot_logits

2024-01-15 10:21:19.631127: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 10:21:19.633085: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 10:21:19.672667: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 10:21:19.674149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 10:21:20.689621: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [6]:
unlabelled_source = "/mnt/input"
labelled_source = "/mnt/labelled"
model_output_file = "/mnt/output/pw_model.keras"
embeddings_dir = "/mnt/embeddings/"
search_results_file = "/mnt/output/search_results.csv"

skip_if_file_exists = True
embedding_model_version = 4

print(Path(unlabelled_source).exists())
print(Path(labelled_source).exists())
print(Path(model_output_file).parent.exists())
print(Path(embeddings_dir).exists())
print(Path(search_results_file).parent.exists())




True
True
True
True
True


In [4]:


if skip_if_file_exists and Path(model_output_file).exists() and Path(model_output_file + '.labels.json').exists():
    classifier_model = model_output_file
    with open(model_output_file + '.labels.json', 'r') as file:
        labels = json.load(file)
else:
    classifier_model, labels = train_and_save(labelled_source, model_output_file, embedding_model_version)

Embedding from Folder of Folders...


  0%|          | 0/16 [00:00<?, ?it/s]2024-01-15 10:21:41.920441: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e5049a16f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-01-15 10:21:41.920501: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-01-15 10:21:42.235458: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-15 10:21:42.241924: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator jax2tf_infer_fn_/assert_equal_1/Assert/AssertGuard/Assert
2024-01-15 10:21:44.823900: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
100%|██████████| 14/14 [00:16<00:00,  1.19s/it]



...embedded dataset in 71.47s...
    found 3 classes.
    class neg / 0 : 16
    class pos / 1 : 16
    class unknown_cgw / 2 : 14
num classes : 3
mean ex / class : 15.333333333333334
min ex / class : 14.0
acc: 0.82, auc_roc: 0.96, cmap: 0.95

neg     , auc_roc: 0.89
test_logits(neg) : 	shape:             (4,)	min:  -0.51	mean:   0.50	max:   1.37	std:   0.76

pos     , auc_roc: 1.00
test_logits(pos) : 	shape:             (4,)	min:  -0.64	mean:   0.40	max:   1.53	std:   0.88

unknown_cgw, auc_roc: 1.00
test_logits(unknown_cgw) : 	shape:             (3,)	min:   1.36	mean:   1.58	max:   1.72	std:   0.16


In [ ]:
if skip_if_file_exists and Path(search_results_file).exists():
    inference_results = pd.read_csv(search_results_file)
else:
    inference_results = process_embeddings(embeddings_dir, classifier_model, search_results_file, labels)

In [ ]:
plot_logits(inference_results, target_class = "pos", target_logit = None)

In [ ]:
display_search_results(inference_results, labels, target_label='pos')